# NBA球员数据 爬虫+数据固化+可视化 

由于腾讯体育网页采用异步加载，选择selenium作为爬虫方案

## 环境配置

In [1]:
import pandas as pd 
import numpy as np
import pymysql
from urllib.request import urlretrieve
import requests
import os 
import time
import threading
from selenium import webdriver  

from selenium.webdriver.chrome.options import Options
chrome_options = Options()
# chrome_options.add_argument('--headless')  # 无头模式 不显示浏览器

## 爬取与解析 selenium+xpath+多线程

按照30支球队，解析球队信息
按照球员解析个人信息
每将数据固化到csv文件，共计应产生1个联盟球队信息文件及30个球队球员信息文件

In [2]:
def team_parse(tid, df):
    
    try:
        browser = webdriver.Chrome(options=chrome_options)
        browser.get('https://sports.qq.com/kbsweb/teams.htm?tid={0}&cid=100000#stats'.format(tid))
        browser.maximize_window()
        time.sleep(5)
    except Exception as e:
        print(e)
        
    team_name = browser.find_elements_by_xpath('//*[@id="team-introduction"]/div/div/div[1]/div')
    team_pic_url = browser.find_elements_by_xpath('//*[@id="team-introduction"]/div/div/img')
    area = browser.find_elements_by_xpath('//*[@id="team-introduction"]/div/div/div[1]/ul/li[1]/span[1]')
    coach = browser.find_elements_by_xpath('//*[@id="team-introduction"]/div/div/div[1]/ul/li[3]/span[2]')
    home_court = browser.find_elements_by_xpath('//*[@id="team-introduction"]/div/div/div[1]/ul/li[4]/span[2]')
    win_num = browser.find_elements_by_xpath('//*[@id="team-introduction"]/div/div/div[2]/div[1]/span[1]/em')
    lose_num = browser.find_elements_by_xpath('//*[@id="team-introduction"]/div/div/div[2]/div[1]/span[2]/em')
    score = browser.find_elements_by_xpath('//*[@id="union-contrast"]/div/div[2]/div[2]/div[1]/div[1]/span')
    rebound = browser.find_elements_by_xpath('//*[@id="union-contrast"]/div/div[2]/div[2]/div[2]/div[1]/span')
    assist = browser.find_elements_by_xpath('//*[@id="union-contrast"]/div/div[2]/div[2]/div[3]/div[1]/span')
    steal = browser.find_elements_by_xpath('//*[@id="union-contrast"]/div/div[2]/div[2]/div[4]/div[1]/span')
    block = browser.find_elements_by_xpath('//*[@id="union-contrast"]/div/div[2]/div[2]/div[5]/div[1]/span')
    
    df.loc[tid, 'team_name'] = [i.text for i in team_name][0]
    df.loc[tid, 'team_pic_url'] = [i.get_attribute('src') for i in team_pic_url][0]
    df.loc[tid, 'area'] = [i.text for i in area][0]
    df.loc[tid, 'coach'] = [i.text for i in coach][0]
    df.loc[tid, 'home_court'] = [i.text for i in home_court][0]
    df.loc[tid, 'win_num'] = [i.text for i in win_num][0]
    df.loc[tid, 'lose_num'] = [i.text for i in lose_num][0]
    df.loc[tid, 'score'] = [i.text for i in score][0]
    df.loc[tid, 'rebound'] = [i.text for i in rebound][0]
    df.loc[tid, 'assist'] = [i.text for i in assist][0]
    df.loc[tid, 'steal'] = [i.text for i in steal][0]
    df.loc[tid, 'block'] = [i.text for i in block][0]
    
    browser.quit()
    return df

In [3]:
def player_parse(tid):
    
    try:
        browser = webdriver.Chrome(options=chrome_options)
        browser.get('https://sports.qq.com/kbsweb/teams.htm?tid={0}&cid=100000#roster'.format(tid))
        browser.maximize_window()
        time.sleep(5)
    except Exception as e:
        print(e)
    df = pd.DataFrame()
    lent = browser.find_elements_by_xpath('//*[@id="teamContent"]/div[3]/div[3]/div/table/tbody')
    player_num = [i.text for i in lent][0].count('\n') + 1  # 通过统计球员标签换了多少行来统计有多少人

    for i in range(player_num):

        player_pic_url = browser.find_elements_by_xpath('//*[@id="teamContent"]/div[3]/div[3]/div/table/tbody/tr[{0}]/td[1]/a/img'.format(i+1))
        player_name = browser.find_elements_by_xpath('//*[@id="teamContent"]/div[3]/div[3]/div/table/tbody/tr[{0}]/td[1]/a/span'.format(i+1))
        player_number = browser.find_elements_by_xpath('//*[@id="teamContent"]/div[3]/div[3]/div/table/tbody/tr[{0}]/td[2]/span'.format(i+1))
        player_roles = browser.find_elements_by_xpath('//*[@id="teamContent"]/div[3]/div[3]/div/table/tbody/tr[{0}]/td[3]/span'.format(i+1))
        player_height = browser.find_elements_by_xpath('//*[@id="teamContent"]/div[3]/div[3]/div/table/tbody/tr[{0}]/td[4]/span'.format(i+1))
        player_weight = browser.find_elements_by_xpath('//*[@id="teamContent"]/div[3]/div[3]/div/table/tbody/tr[{0}]/td[5]/span'.format(i+1))
        play_years = browser.find_elements_by_xpath('//*[@id="teamContent"]/div[3]/div[3]/div/table/tbody/tr[{0}]/td[6]/span'.format(i+1))


        df.loc[i, 'player_pic_url'] = [i.get_attribute('src') for i in player_pic_url][0]
        df.loc[i, 'player_name'] = [i.text for i in player_name][0]
        df.loc[i, 'player_number'] = [i.text for i in player_number][0]
        df.loc[i, 'player_roles'] = [i.text for i in player_roles][0]
        df.loc[i, 'player_height'] = [i.text for i in player_height][0]
        df.loc[i, 'player_weight'] = [i.text for i in player_weight][0]
        df.loc[i, 'play_years'] = [i.text for i in play_years][0]
        
    team_name = browser.find_elements_by_xpath('//*[@id="team-introduction"]/div/div/div[1]/div')
    df.to_csv('players_{0}.csv'.format([i.text for i in team_name][0]), encoding='utf-8-sig')
    print('-'*20 + '第{0}支球队全体球员数据已写入csv文件'.format(tid) + '-'*20)
    browser.quit()

调用爬取函数

In [4]:
# for team_id in range(24, 31):
#     team_df = pd.DataFrame()
#     print('-'*20 + '正在下载第{0}支球队综合数据'.format(team_id) + '-'*20)
#     team_parse(team_id, team_df).to_csv('teams_联盟_1.csv',mode='a',encoding='utf-8-sig')
#     print('-'*20 + '第{0}支球队综合数据已写入csv文件'.format(team_id) + '-'*20)
    
#     print('-'*20 + '正在下载第{0}支球队球员数据'.format(team_id) + '-'*20)
#     player_parse(team_id)

多线程爬取 - 更快更高效

In [5]:
def multi_parse(start_id, end_id):
    for team_id in range(start_id, end_id):

        team_df = pd.DataFrame()
        print('-'*20 + '正在下载第{0}支球队综合数据'.format(team_id) + '-'*20)
        team_parse(team_id, team_df).to_csv('teams_multi.csv',mode='a',encoding='utf-8-sig')
        print('-'*20 + '第{0}支球队综合数据已写入csv文件'.format(team_id) + '-'*20)

        print('-'*20 + '正在下载第{0}支球队球员数据'.format(team_id) + '-'*20)
        print(team_id)
        player_parse(team_id)


threads=[]
 
t1 = threading.Thread(target=multi_parse,args=(1,8))
threads.append(t1)
 
t2 = threading.Thread(target=multi_parse,args=(8,16))
threads.append(t2)
 
t3 = threading.Thread(target=multi_parse,args=(16,24))
threads.append(t3)
 
t4 = threading.Thread(target=multi_parse,args=(24,31))
threads.append(t4)

# join() 这会阻塞调用线程，直到其join（）方法被调用的线程终止 - 此处可以让主程序挂起，以正常打印日志

# for t in threads:
#     t.start()
# for t in threads:
#     t.join()
    
print('全部爬取完毕.')

全部爬取完毕.


爬取结束，数据已被存储到csv文件中，下一步运用pymysql将数据批量固化至mysql数据库中。

## 数据固化 pymysql

In [6]:
datadir = r'C:\Users\Jianming\Desktop\技术栈\爬虫\NBA_spider\爬取结果\players'
filelist = os.listdir(datadir)
filelist

['players_76人.csv',
 'players_公牛.csv',
 'players_凯尔特人.csv',
 'players_勇士.csv',
 'players_国王.csv',
 'players_太阳.csv',
 'players_奇才.csv',
 'players_尼克斯.csv',
 'players_开拓者.csv',
 'players_快船.csv',
 'players_掘金.csv',
 'players_森林狼.csv',
 'players_步行者.csv',
 'players_活塞.csv',
 'players_湖人.csv',
 'players_火箭.csv',
 'players_灰熊.csv',
 'players_热火.csv',
 'players_爵士.csv',
 'players_独行侠.csv',
 'players_猛龙.csv',
 'players_篮网.csv',
 'players_老鹰.csv',
 'players_雄鹿.csv',
 'players_雷霆.csv',
 'players_马刺.csv',
 'players_骑士.csv',
 'players_魔术.csv',
 'players_鹈鹕.csv',
 'players_黄蜂.csv']

先建立NBA数据库用于存储

In [7]:
conn = pymysql.connect(user='root', passwd='xjm970722', db='test_db')
cursor = conn.cursor()
cursor.execute('CREATE DATABASE IF NOT EXISTS NBA_db DEFAULT CHARSET utf8 COLLATE utf8_general_ci;')
cursor.close()  # 先关闭游标
conn.close()  # 再关闭数据库连接
print('已创建数据库 NBA_db')

已创建数据库 NBA_db


批量创建数据表的函数构造

In [8]:
def insert_table(table):

    conn = pymysql.connect(user='root', passwd='xjm970722', db='NBA_db')
    cursor = conn.cursor()
    cursor.execute('drop table if exists {0}'.format(table))
    sql = """CREATE TABLE IF NOT EXISTS `{0}`(
                `player_name` varchar(255) NOT NULL,
                `player_pic_url` varchar(255) NOT NULL,
                `player_number` int(11) NOT NULL,
                `player_roles` varchar(255) NOT NULL,
                `player_weight` float NOT NULL,
                `player_height` float NOT NULL,
                `play_years` int(11),
                PRIMARY KEY(`player_name`)
            ) ENGINE=InnoDB  DEFAULT CHARSET=utf8 AUTO_INCREMENT=0
            """.format(table)

    cursor.execute(sql)
    print('创建数据表: {0}'.format(table))

    data = pd.read_csv(r'C:\Users\Jianming\Desktop\技术栈\爬虫\NBA_spider\爬取结果\players\{0}.csv'.format(table)).iloc[:, 1:]
    data.play_years.replace('-', '0', inplace = True)
    data_insert = data.values.tolist()  # 准备数据，即将插入数据表

    cursor.executemany('INSERT INTO {0} VALUES(%s, %s, %s, %s, %s, %s, %s)'.format(table), data_insert)
    
    print('已插入数据到: {0}'.format(table))

    cursor.close()
    conn.commit()  # 将更改保存到数据库
    conn.close()

批量由csv插入数据库表
(联盟球队综合只有一张表，手动导入)

In [9]:
# for table in filelist:
#     table = table.split('.')[0]
#     insert_table(table)
# print('球员数据全部存储完毕')

发现需要构建一张包括所有球员的大表，同时新增一列 player_team 表示所属球队 

In [ ]:
def insert_table_total(table):

    conn = pymysql.connect(user='root', passwd='xjm970722', db='nba_db')
    cursor = conn.cursor()
#     cursor.execute('drop table if exists total_players')
    sql = """CREATE TABLE IF NOT EXISTS `total_players`(
                `player_pic_url` varchar(255) NOT NULL,
                `player_name` varchar(255) NOT NULL,
                `player_number` int(11) NOT NULL,
                `player_roles` varchar(255) NOT NULL,
                `player_weight` float NOT NULL,
                `player_height` float NOT NULL,
                `play_years` int(11),
                `player_team` varchar(255) NOT NULL
            )
            """

    cursor.execute(sql)

    data = pd.read_csv(r'C:\Users\Jianming\Desktop\技术栈\爬虫\NBA_spider\爬取结果\players\{0}.csv'.format(table)).iloc[:, 1:]
    data.play_years.replace('-', '0', inplace = True)
    data.loc[:, 'player_team'] = table.split('_')[1]
    data_insert = data.values.tolist()  # 准备数据，即将插入数据表

    cursor.executemany('INSERT INTO total_players VALUES(%s, %s, %s, %s, %s, %s, %s, %s)', data_insert)
    
    print('已插入{0}数据到total_players'.format(table))

    cursor.close()
    conn.commit()  # 将更改保存到数据库
    conn.close()
    

In [18]:
# for table in filelist:
#     table = table.split('.')[0]
#     insert_table_total(table)
# print('球员数据大表生成完毕')

球员数据大表生成完毕


## 可视化分析 Tableau 

可以根据我们之前爬取的球队队标，team_pic_url下载对应图片

In [11]:
# url_list = pd.read_csv(r'C:/Users/Jianming/Desktop/技术栈/爬虫/NBA_spider/爬取结果/teams/teams_联盟.csv').team_pic_url.to_list()
# os.makedirs('./爬取结果/pics/', exist_ok=True)

# i = 1
# for url in url_list:
#     urlretrieve(url, './爬取结果/pics/pic_{0}.png'.format(i))
#     i += 1

同理下载球员图片
此处不需要全部下载